<a href="https://colab.research.google.com/github/SaintFresh/GFPGAN-SF/blob/main/GFPGAN_SF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GFPGAN

## CLONE & SETUP

In [ ]:
# Clone GFPGAN and enter the GFPGAN folder
%cd /content/drive/MyDrive
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr

# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib

# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop

# If you want to enhance the background (non-face) regions with Real-ESRGAN,
# you also need to install the realesrgan package
!pip install realesrgan  # used for enhancing the background (non-face) regions

# Download the pre-trained model
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
# Now we use the V1.3 model for the demo
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

## UPLOAD

In [ ]:
# upload your own images
import os
from google.colab import files
import shutil

upload_folder = 'inputs/upload'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

## INFERENCE


### FIX TORCHVISION ERROR

In [ ]:
import os
from shutil import copyfile

file_path = '/usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py'

try:
    # Create a backup of the original file
    backup_path = file_path + '.bak'
    copyfile(file_path, backup_path)
    print(f"Backup created: {backup_path}")

    # Check if the file exists
    if os.path.isfile(file_path):
        # Open the file in read-write mode and replace the import
        with open(file_path, 'r+') as file:
            content = file.read()
            new_content = content.replace(
                'from torchvision.transforms.functional_tensor import rgb_to_grayscale',
                'from torchvision.transforms.functional import rgb_to_grayscale'
            )

            # Reset file pointer to the beginning and write the modified content
            file.seek(0)
            file.write(new_content)
            file.truncate()  # Remove any remaining content
            print("File revised successfully!")
    else:
        raise FileNotFoundError(f"File not found: {file_path}")  # Raise a specific error

except FileNotFoundError as e:
    print(f"Error: {e}")  # Print the error message for FileNotFoundError
except PermissionError as e:
    print(f"Error: {e}. You may need to run this with sudo permissions.")
    # Suggest running with elevated permissions for PermissionError
except Exception as e:
    print(f"An unexpected error occurred: {e}")  # Generic error handling for other exceptions

finally:
    # Change directory regardless of success or error
    %cd /content/drive/MyDrive/GFPGAN

### RUN INFERENCE

In [ ]:
# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions
# You can find the different models in https://github.com/TencentARC/GFPGAN#european_castle-model-zoo

!rm -rf results

!python inference_gfpgan.py -i inputs/upload -o results -v 1.4 -s 2 --bg_upsampler realesrgan --ext png


# !python inference_gfpgan.py -i inputs/upload -o results -v RestoreFormer -s 2 --bg_upsampler realesrgan --bg_tile 400
# !python inference_gfpgan.py -i results/restored_imgs -o results -v 1.4 -s 4 --bg_upsampler realesrgan --bg_tile 200
# !python inference_gfpgan.py -i inputs/upload -o results -v 1.4 -s 4 --bg_upsampler realesrgan --bg_tile 200

# Usage: python inference_gfpgan.py -i inputs/whole_imgs -o results -v 1.3 -s 2 [options]...
#
#  -h                   show this help
#  -i input             Input image or folder. Default: inputs/whole_imgs
#  -o output            Output folder. Default: results
#  -v version           GFPGAN model version. Option: 1 | 1.2 | 1.3 | 1.4 | RestoreFormer. Default: 1.3
#  -s upscale           The final upsampling scale of the image. Default: 2
#  -bg_upsampler        background upsampler. Default: realesrgan
#  -bg_tile             Tile size for background sampler, 0 for no tile during testing. Default: 400
#  -suffix              Suffix of the restored faces
#  -only_center_face    Only restore the center face
#  -aligned             Input are aligned faces
#  -ext                 Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto

!ls results

## DOWNLOAD

In [ ]:
# prompt: zip all the files in all_res = 'results/restored_imgs', then download that .zip
from google.colab import files

all_res = 'results/restored_imgs'
!zip -r download.zip $all_res
files.download("download.zip")

# ESRGAN

## CLONE

In [ ]:
#SETUP

#Clone repository and navigate to directory
%cd /content/drive/MyDrive/Real-ESRGAN
!rm -rf Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN/

## SETUP

In [ ]:
#Install dependencies
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt

#Setup environment
!python setup.py develop

import shutil

# Source path
source_path = '/content/inference_realesrgan.py'

# Destination path
destination_path = '/content/drive/MyDrive/Real-ESRGAN/Real-ESRGAN'

shutil.move(source_path, destination_path)

## UPLOAD

In [ ]:
#UPLOAD

import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

## INFERENCE

In [ ]:
#INFERENCE

#!python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 6 --face_enhance --tile 200

!python inference_realesrgan.py -n RealESRGAN_x4plus -i /content/drive/MyDrive/GFPGAN/inputs/upload --outscale 6 --face_enhance --tile 200

#usage: inference_realesrgan.py [-h] [-i INPUT] [-n MODEL_NAME] [-o OUTPUT] [-dn DENOISE_STRENGTH]
#                               [-s OUTSCALE] [--model_path MODEL_PATH] [--suffix SUFFIX] [-t TILE]
#                               [--tile_pad TILE_PAD] [--pre_pad PRE_PAD] [--face_enhance] [--fp32]
#                               [--alpha_upsampler ALPHA_UPSAMPLER] [--ext EXT] [-g GPU_ID]
#
#options:
#  -h.       | --help            show this help message and exit
#  -i INPUT  | --input INPUT
#                        Input image or folder
#  -n MODEL_NAME  | --model_name MODEL_NAME
#                        Model names: RealESRGAN_x4plus | RealESRNet_x4plus |
#                        RealESRGAN_x4plus_anime_6B | RealESRGAN_x2plus | realesr-animevideov3 |
#                        realesr-general-x4v3
#  -o OUTPUT, --output OUTPUT
#                        Output folder
#  -dn DENOISE_STRENGTH, --denoise_strength DENOISE_STRENGTH
#                        Denoise strength. 0 for weak denoise (keep noise), 1 for strong denoise
#                        ability. Only used for the realesr-general-x4v3 model
#  -s OUTSCALE, --outscale OUTSCALE
#                        The final upsampling scale of the image
#  --model_path MODEL_PATH
#                        [Option] Model path. Usually, you do not need to specify it
#  --suffix SUFFIX       Suffix of the restored image
#  -t TILE, --tile TILE  Tile size, 0 for no tile during testing
#  --tile_pad TILE_PAD   Tile padding
#  --pre_pad PRE_PAD     Pre padding size at each border
#  --face_enhance        Use GFPGAN to enhance face
#  --fp32                Use fp32 precision during inference. Default: fp16 (half precision).
#  --alpha_upsampler ALPHA_UPSAMPLER
#                        The upsampler for the alpha channels. Options: realesrgan | bicubic
#  --ext EXT             Image extension. Options: auto | jpg | png, auto means using the same
#                        extension as inputs
#  -g GPU_ID, --gpu-id GPU_ID
#                        gpu device to use (default=None) can be 0,1,2 for multi-gpu

## DOWNLOAD

In [ ]:
# Download the results
zip_filename = 'Real-ESRGAN_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} results/*")
files.download(zip_filename)

results_filename = 'Real-ESRGAN_result.gz'
if os.path.exists(results_filename):
  os.remove(results_filename)
os.system(f"zip -r -j {results_filename} results/*")
files.download(results_filename)

!ls /content/Real-ESRGAN/results/*